# Notebook para tratamento dos dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

#style.use('seaborn')
%matplotlib inline


In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from sklearn import preprocessing, metrics
# !pip install -U scikit-fuzzy
import sys
import skfuzzy as fuzz
sys.setrecursionlimit(1000)

In [3]:
data_column_names = ["Date", "Time", "Epoch", "Moteid", "Temperature", "Humidity", "Light", "Voltage"]
dplot = pd.read_csv("../data.txt", sep = ' ', names = data_column_names)

s = 23.0
S25df = dplot[dplot['Moteid']==25]
S27df = dplot[dplot['Moteid']==27]

In [5]:
S23df = pd.read_csv('dfImputation.csv')
S23df.shape,S25df.shape,S27df.shape

((65629, 8), (53175, 8), (37628, 8))

In [14]:
S23df['3'] = s
S23df.head()

,0,1,2,3,4,5,6,7
0,2004-03-31,03:38:15.757551,0,23.0,0.0,-3.91901,11.04,2.03397
1,2004-02-28,00:59:16.02785,1,23.0,0.0,37.09330,45.08,2.69964
2,2004-04-02,14:00:28.893529,2,23.0,-38.4,23.38000,625.60,1.92207
3,2004-02-28,01:03:16.33393,3,23.0,0.0,38.46290,45.08,2.68742
4,2004-04-02,14:01:33.462872,4,23.0,NaN,NaN,NaN,1.99973


In [15]:
outliersIL = []
outliersILValues = []
for i, row in S25df.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL.append(i)
        outliersILValues.append(row['Temperature'])
print(len(outliersIL))


23903


In [16]:
# Removendo outliers no Sensor
passo3= S25df.copy()
print(passo3.shape)
passo3= passo3.drop(outliersIL)
print(passo3.shape)
passo3.to_csv('S25FINAL.csv')

(53175, 8)
(29272, 8)


In [18]:
outliersIL2 = []
outliersILV2 = []
for i, row in S27df.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL2.append(i)
        outliersILV2.append(row['Temperature'])
print(len(outliersIL2))


14770


In [19]:
# Removendo outliers no S2
passo3_3= S27df.copy()
print(passo3_3.shape)
passo3_3= passo3_3.drop(outliersIL2)
print(passo3_3.shape)
passo3_3.to_csv('S27FINAL.csv')

(37628, 8)
(22858, 8)


In [20]:
outliersIL_3 = []
outliersILV_3 = []
for i, row in S23df.iterrows():
    if row['4'] >=26.2:
        outliersIL_3.append(i)
        outliersILV_3.append(row['4'])
print(len(outliersIL_3))


0


In [21]:
# Removendo outliers no sensor
passo3_2= S23df.copy()
print(passo3_2.shape)
passo3_2= passo3_2.drop(outliersIL_3)
print(passo3_2.shape)
passo3_2.to_csv('S23FINAL.csv')

(65629, 8)
(65629, 8)


In [22]:
S23df_ = pd.read_csv('S23FINAL.csv')
S27df_ = pd.read_csv('S27FINAL.csv')
S25df_ = pd.read_csv('S25FINAL.csv')
S23df_['3'] = s
S23df_.shape, S25df_.shape, S27df_.shape

((65629, 9), (29272, 9), (22858, 9))

In [23]:
S23df_.head()

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,2004-03-31,03:38:15.757551,0,23.0,0.0,-3.91901,11.04,2.03397
1,1,2004-02-28,00:59:16.02785,1,23.0,0.0,37.09330,45.08,2.69964
2,2,2004-04-02,14:00:28.893529,2,23.0,-38.4,23.38000,625.60,1.92207
3,3,2004-02-28,01:03:16.33393,3,23.0,0.0,38.46290,45.08,2.68742
4,4,2004-04-02,14:01:33.462872,4,23.0,NaN,NaN,NaN,1.99973


# Sistema Fuzzy

In [24]:
# Modelo Fuzzy
temp = np.arange(-26.2, 26.2, 0.1)
tempF = np.arange(-26.2, 26.2, 0.1)

# Generate fuzzy membership functions
temp_lo = fuzz.trimf(temp, [-26.2, 0.0, 5.0])  
temp_md = fuzz.trimf(temp, [0.0, 5.0, 20.0])      
temp_hi = fuzz.trimf(temp, [17.20, 22.8, 26.2]) 


mfx = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2])


tempFinal_lo = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2]) 
tempFinal_md = fuzz.trimf(tempF, [0, 5.0, 20.0]) 
tempFinal_hi = fuzz.trimf(tempF, [17.20, 22.8, 26.2])


In [25]:
def fuzification(i, data, data1, data2): 
    if data['3'].ix[i]== s:
        temp_level_lo0 = fuzz.interp_membership(temp, temp_lo,  data1['Temperature'][i]) # ex: [52, [51, 52]]
        temp_level_md0 = fuzz.interp_membership(temp, temp_md,  data1['Temperature'][i])
        temp_level_hi0 = fuzz.interp_membership(temp, temp_hi,  data1['Temperature'][i])

        temp_level_lo1 = fuzz.interp_membership(temp, temp_lo,  data2['Temperature'][i])
        temp_level_md1 = fuzz.interp_membership(temp, temp_md,  data2['Temperature'][i])
        temp_level_hi1 = fuzz.interp_membership(temp, temp_hi,  data2['Temperature'][i])

        active_rule1_0 = np.fmin(temp_level_lo0, temp_level_lo1)
        active_rule1_1 = np.fmin(temp_level_lo0, temp_level_md1)
        active_rule1_2 = np.fmin(temp_level_lo0, temp_level_hi1)
        active_rule1_3 = np.fmin(temp_level_md0, temp_level_lo1)
        active_rule1_4 = np.fmin(temp_level_md0, temp_level_md1)
        active_rule1_5 = np.fmin(temp_level_md0, temp_level_hi1)
        active_rule1_6 = np.fmin(temp_level_hi0, temp_level_lo1)
        active_rule1_7 = np.fmin(temp_level_hi0, temp_level_md1)
        active_rule1_8 = np.fmin(temp_level_hi0, temp_level_hi1)


        active_rule1_Final_0 = np.fmax(active_rule1_0, tempFinal_lo)
        active_rule1_Final_1 = np.fmax(active_rule1_1, tempFinal_lo)
        active_rule1_Final_2 = np.fmax(active_rule1_2, tempFinal_md)
        active_rule1_Final_3 = np.fmax(active_rule1_3, tempFinal_lo)
        active_rule1_Final_4 = np.fmax(active_rule1_4, tempFinal_hi)
        active_rule1_Final_5 = np.fmax(active_rule1_5, tempFinal_hi)
        active_rule1_Final_6 = np.fmax(active_rule1_6, tempFinal_md)
        active_rule1_Final_7 = np.fmax(active_rule1_7, tempFinal_hi)
        active_rule1_Final_8 = np.fmax(active_rule1_8, tempFinal_hi)

        r1_0 = np.fmax(active_rule1_Final_8, np.fmax(active_rule1_Final_7, np.fmax(active_rule1_Final_6, active_rule1_Final_5) ) )  
        r1_1 = np.fmax(active_rule1_Final_5, np.fmax(active_rule1_Final_3, np.fmax(active_rule1_Final_2, np.fmax(active_rule1_Final_1, active_rule1_Final_0)) ) )  
        aggregated = np.fmax(r1_0,r1_1)
        return aggregated  


In [26]:
def defuzz(x, mfx, mode):
    mode = mode.lower()
    x = x.ravel()
    mfx = mfx.ravel()
    n = len(x)
    assert n == len(mfx), 'Length of x and fuzzy membership function must be \
                          identical.'

    if 'Centroid' in mode or 'bisector' in mode:
        tot_area = mfx.sum()
        assert tot_area != 0, 'Total area is zero in defuzzification!'

        if 'Centroid' in mode:
            return centroid(x, mfx)

        elif 'bisector' in mode:
            tmp = 0
            for k in range(n):
                tmp += mfx[k]
                if tmp >= tot_area / 2.:
                    return x[k]

    elif 'mom' in mode:
        return np.mean(x[mfx == mfx.max()])

    elif 'som' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).min()][0]

    elif 'lom' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).max()][0]

    else:
        raise ValueError('The input for `mode`, %s, was incorrect.' % (mode))



In [27]:
aux = 0.0
for i, row in S23df_.iterrows():
        if S23df_['4'][i] == 0.0:            
            try:
                S23df_.at[i, '4'] = defuzz(tempF,fuzification(i, S23df_,  S25df_,S27df_ ), 'lom')
                aux = defuzz(tempF,fuzification(i, S23df_, S25df_,S27df_ ), 'lom')
            except:
                S23df_.at[i, '4'] = aux             
S23df_.tail()


,Unnamed: 0,0,1,2,3,4,5,6,7
65624,65624,2004-03-21,19:04:02.05067,65531,23.0,24.2612,46.6887,9.2,2.35683
65625,65625,2004-03-21,19:04:30.546777,65532,23.0,24.2220,46.4265,9.2,2.34751
65626,65626,2004-03-21,19:05:08.368529,65533,23.0,24.2612,46.7542,9.2,2.35683
65627,65627,2004-03-21,19:05:30.263677,65534,23.0,24.1828,46.7215,9.2,2.35683
65628,65628,2004-03-21,19:05:56.944589,65535,23.0,24.1632,46.6887,9.2,2.35683


In [28]:
print(S23df_.shape)
S23df_.to_csv('finalsensor.csv', index=False)

(65629, 9)


# Final